In [2]:
import os

In [12]:
os.getcwd()

'/Users/maciasalvasalva/Desktop/UNED/Descubrimiento de informacion en textos/Entregas/PEC3'

In [18]:
path = os.getcwd() + "/Corpus-representacion"
# Obtener el listado
listado = os.listdir(path)

corpus_dir = [] # Guardamos el directorio para cada fichero
for elemento in listado:
    corpus_dir.append(path +'/'+elemento)

In [22]:
corpus_dir

['/Users/maciasalvasalva/Desktop/UNED/Descubrimiento de informacion en textos/Entregas/PEC3/Corpus-representacion/talk.politics.mideast',
 '/Users/maciasalvasalva/Desktop/UNED/Descubrimiento de informacion en textos/Entregas/PEC3/Corpus-representacion/rec.autos',
 '/Users/maciasalvasalva/Desktop/UNED/Descubrimiento de informacion en textos/Entregas/PEC3/Corpus-representacion/comp.sys.mac.hardware',
 '/Users/maciasalvasalva/Desktop/UNED/Descubrimiento de informacion en textos/Entregas/PEC3/Corpus-representacion/rec.sport.hockey',
 '/Users/maciasalvasalva/Desktop/UNED/Descubrimiento de informacion en textos/Entregas/PEC3/Corpus-representacion/comp.sys.ibm.pc.hardware',
 '/Users/maciasalvasalva/Desktop/UNED/Descubrimiento de informacion en textos/Entregas/PEC3/Corpus-representacion/sci.electronics',
 '/Users/maciasalvasalva/Desktop/UNED/Descubrimiento de informacion en textos/Entregas/PEC3/Corpus-representacion/talk.politics.guns']

In [39]:
def read_file(filename):
    with open(filename, "r") as archivo:
        lineas = archivo.readlines()
        start_read_file = False
        for linea in lineas:
            if linea.startswith("Lines:"):
                start_read_file = True
                continue
            if start_read_file:
                print(linea.strip())
    

for dir in corpus_dir:
    print("**** "+dir+" ****")
    corpus_files = os.listdir(dir)
    for corpus in corpus_files:
        print(dir +'/'+corpus)
        read_file(dir +'/'+corpus)
        break


**** /Users/maciasalvasalva/Desktop/UNED/Descubrimiento de informacion en textos/Entregas/PEC3/Corpus-representacion/talk.politics.mideast ****
/Users/maciasalvasalva/Desktop/UNED/Descubrimiento de informacion en textos/Entregas/PEC3/Corpus-representacion/talk.politics.mideast/75406
Message-ID: <FLAX.93Apr6125933@frej.teknikum.uu.se>
References: <1993Mar30.142700.543@vms.huji.ac.il> <FLAX.93Apr3142133@frej.teknikum.uu.se>
<FLAX.93Apr5224449@frej.teknikum.uu.se>
<1993Apr5.221759.28472@thunder.mcrcim.mcgill.edu>
NNTP-Posting-Host: frej.teknikum.uu.se
In-reply-to: hasan@McRCIM.McGill.EDU 's message of Mon, 5 Apr 93 22:17:59 GMT

In article <1993Apr5.221759.28472@thunder.mcrcim.mcgill.edu> hasan@McRCIM.McGill.EDU  writes:

[ stuff deleted ]
|> I wrote:
|> Are you calling names, or giving me a title? If the first, read your
|> paragraph above, if not I accept the title, in order to let you get into the
|> um, well, debate again.

Hasan replies:
I didnot know that "Master of wisdom" can be "

In [8]:
corpus_dir

['talk.politics.mideast',
 'rec.autos',
 'comp.sys.mac.hardware',
 'rec.sport.hockey',
 'comp.sys.ibm.pc.hardware',
 'sci.electronics',
 'talk.politics.guns']

In [3]:
def eliminar_cabecera(texto):
    # Dividir el texto en líneas
    lineas = texto.splitlines()
    
    # Buscar la línea que contiene 'Lines:'
    for i, linea in enumerate(lineas):
        if linea.startswith("Lines:"):
            # Cortar el texto desde la línea siguiente
            contenido = "\n".join(lineas[i + 1:])
            return contenido

    # Si no se encuentra 'Lines:', devolver el texto completo
    return texto
